In [1]:
import logging

logging.basicConfig(level=logging.DEBUG)

In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [3]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath(__name__))))
SOURCE_DIR
sys.path.insert(0, SOURCE_DIR)

In [4]:
%%time

import malaya
malaya

DEBUG:tensorflow:Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7
DEBUG:git.cmd:Popen(['git', 'version'], cwd=/home/husein/dev/malaya/misc/normalize-twitter, universal_newlines=False, shell=None, istream=None)
DEBUG:git.cmd:Popen(['git', 'version'], cwd=/home/husein/dev/malaya/misc/normalize-twitter, universal_newlines=False, shell=None, istream=None)


CPU times: user 3.49 s, sys: 3.23 s, total: 6.73 s
Wall time: 2.61 s


<module 'malaya' from '/home/husein/dev/malaya/malaya/__init__.py'>

In [5]:
lm = malaya.language_model.kenlm(model = 'bahasa-wiki-news')

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /huseinzol05/language-model-bahasa-wiki-news/resolve/main/model.klm HTTP/1.1" 302 0


In [6]:
stemmer = malaya.stem.deep_model(model = 'noisy')

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /huseinzol05/stem-v2-lstm-bahdanau-noisy/resolve/main/model.pb HTTP/1.1" 302 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /huseinzol05/bpe/resolve/main/stemmer-v2.yttm HTTP/1.1" 200 0
DEBUG:malaya_boilerplate.frozen_graph:use_tensorrt: False
DEBUG:malaya_boilerplate.frozen_graph:tensorrt_precision_mode: FP32
DEBUG:malaya_boilerplate.frozen_graph:precision_mode: FP32
DEBUG:malaya_boilerplate.frozen_graph:t5_graph: False
DEBUG:malaya_boilerplate.frozen_graph:glowtts_graph: False
DEBUG:malaya_boilerplate.frozen_graph:glowtts_multispeaker_graph: False
2022-12-09 15:25:18.532818: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instruct

In [7]:
corrector = malaya.spelling_correction.probability.load(language_model = lm, replace_augmentation = True,
                                                       stemmer = stemmer, maxlen = 12, minlen = 2)

INFO:malaya_boilerplate.huggingface:downloading frozen huseinzol05/v27-preprocessing/bm_1grams.json
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /huseinzol05/v27-preprocessing/resolve/main/bm_1grams.json HTTP/1.1" 200 0


In [8]:
normalizer = malaya.normalize.normalizer(corrector, stemmer, date = False)

In [9]:
# normalizer._tokenizer(t)

In [10]:
segmenter = malaya.segmentation.huggingface()
segmenter_func = lambda x: segmenter.generate([x], max_length = 256)[0]

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /mesolitica/finetune-segmentation-t5-tiny-standard-bahasa-cased/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /mesolitica/finetune-segmentation-t5-tiny-standard-bahasa-cased/resolve/main/config.json HTTP/1.1" 200 0


In [11]:
text_scorer = lambda x: lm.score(x)

In [12]:
t = '14:14hrs Kerja penyelenggaraan KM 83.5- KM 84.5 arah Utara dari Ayer Hitam ke Yong Peng. Lorong kiri ditutup. Trafi https://t.co/iP2jEdY4Ib'

In [13]:
t = 'Tok aza.....semoga tok aza dapat jaga undang2 mcm mana tok aza jaga dewan masa jd speaker....hara\npan tgi pada tok aza skrg ni'

In [14]:
segmenter.generate([t], max_length = 512)

DEBUG:malaya.torch_model.huggingface:generate, initial_text: segmentasi: 
DEBUG:malaya.torch_model.huggingface:generate, strings: ['Tok aza.....semoga tok aza dapat jaga undang2 mcm mana tok aza jaga dewan masa jd speaker....hara\npan tgi pada tok aza skrg ni']
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


['Tok aza.....semoga tok aza dapat jaga undang2 mcm mana tok aza jaga dewan masa jd speaker....hara pan tgi pada tok aza skrg ni']

In [15]:
true_case = malaya.true_case.huggingface()

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /mesolitica/finetune-true-case-t5-tiny-standard-bahasa-cased/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /mesolitica/finetune-true-case-t5-tiny-standard-bahasa-cased/resolve/main/config.json HTTP/1.1" 200 0


In [16]:
t = 'JOHOR BAHRU, 27 April- 903,000 ketua isi rumah di Johor bakal menerima Bantuan Beras Bangsa Johor yang mula diedar https://t.co/IBGR0fYM93'

In [17]:
import re
from malaya.text.regex import (
    _past_date_string,
    _now_date_string,
    _future_date_string,
    _yesterday_tomorrow_date_string,
    _depan_date_string,
    _expressions,
    _left_datetime,
    _right_datetime,
    _today_time,
    _left_datetodaytime,
    _right_datetodaytime,
    _left_yesterdaydatetime,
    _right_yesterdaydatetime,
    _left_yesterdaydatetodaytime,
    _right_yesterdaydatetodaytime,
)
from malaya.text.normalization import (
    _remove_postfix,
    _normalize_title,
    _is_number_regex,
    _string_to_num,
    _replace_compound,
    cardinal,
    digit_unit,
    rom_to_int,
    ordinal,
    fraction,
    money,
    ignore_words,
    digit,
    unpack_english_contractions,
    repeat_word,
    replace_laugh,
    replace_mengeluh,
    replace_betul,
    digits,
    normalize_numbers_with_shortform,
)

In [19]:
word = '903,000'
word_lower = word.lower()
(len(re.findall(r'(.)\1{1}', word))
and not word[0].isupper()
and not word_lower.startswith('ke-')
and not len(re.findall(_expressions['email'], word))
and not len(re.findall(_expressions['url'], word))
and not len(re.findall(_expressions['hashtag'], word))
and not len(re.findall(_expressions['phone'], word))
and not len(re.findall(_expressions['money'], word))
and not len(re.findall(_expressions['date'], word))
and not len(re.findall(_expressions['ic'], word))
and not len(re.findall(_expressions['user'], word))
and not len(re.findall(_expressions['number'], word))
and not _is_number_regex(word))

False

In [20]:
# t_ = true_case.generate([t], max_length = 512)[0]
# t_

In [21]:
normalized = normalizer.normalize(
            t, normalize_hingga = False, normalize_cardinal = False,
            normalize_ordinal = False, normalize_pada_hari_bulan = False,
            normalize_fraction = False, normalize_money = False, normalize_date = False,
            normalize_time = False, normalize_ic = False,
            normalize_units = False,
            normalize_url = False, normalize_percent=False, normalize_telephone = False,
            text_scorer = text_scorer, normalize_elongated = True,
            not_a_word_threshold = 1e-9,
        )['normalize']
normalized

INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): raw.githubusercontent.com:443
DEBUG:urllib3.connectionpool:https://raw.githubusercontent.com:443 "GET /huseinzol05/malay-dataset/master/dictionary/emoji/demoji.json HTTP/1.1" 200 52544
DEBUG:malaya.normalize:before expand_contractions: JOHOR BAHRU, 27 April- 903,000 ketua isi rumah di Johor bakal menerima Bantuan Beras Bangsa Johor yang mula diedar https://t.co/IBGR0fYM93
DEBUG:malaya.normalize:after expand_contractions: JOHOR BAHRU, 27 April- 903,000 ketua isi rumah di Johor bakal menerima Bantuan Beras Bangsa Johor yang mula diedar https://t.co/IBGR0fYM93
DEBUG:malaya.normalize:tokenized: ['JOHOR', 'BAHRU', ',', '27', 'April', '-', '903,000', 'ketua', 'isi', 'rumah', 'di', 'Johor', 'bakal', 'menerima', 'Bantuan', 'Beras', 'Bangsa', 'Johor', 'yang', 'mula', 'diedar', 'https://t.co/IBGR0fYM93']
DEBUG:malaya.normalize:before normalize_elongated: JOH

DEBUG:malaya.normalize:index: 14, word: Bantuan, score: 0.0007869930162521621, text_scorer is not None
DEBUG:malaya.normalize:index: 14, word: Bantuan, condition not in money and date
DEBUG:malaya.normalize:index: 14, word: Bantuan, condition text_scorer is not None
DEBUG:malaya.normalize:index: 14, word: Bantuan, lower: bakal menerima bantuan Beras Bangsa , normal: bakal menerima Bantuan Beras Bangsa , lower_score: -20.368301391601562, title_score: -19.99850082397461, normal_score: -19.99850082397461, upper_score: -25.98023223876953
DEBUG:malaya.normalize:index: 14, word: Bantuan, condition titled
DEBUG:malaya.normalize:index: 15, word: Beras, queue: ['JOHOR', 'BAHRU', ',', '27', 'April', '-', '903,000', 'ketua', 'isi', 'rumah', 'di', 'Johor', 'bakal', 'menerima', 'Bantuan']
DEBUG:malaya.normalize:index: 15, word: Beras, score: 0.0036940168338038225, text_scorer is not None
DEBUG:malaya.normalize:index: 15, word: Beras, condition not in money and date
DEBUG:malaya.normalize:index: 15,

'JOHOR BAHRU , 27 April - 903,000 ketua isi rumah di Johor bakal menerima Bantuan Beras Bangsa Johor yang mula diedar https://t.co/IBGR0fYM93'

In [22]:
from malaya.text.tatabahasa import (
    date_replace,
    consonants,
    sounds,
    bulan,
)
from malaya.text.function import (
    is_laugh,
    is_mengeluh,
    multireplace,
    case_of,
    PUNCTUATION,
)

In [24]:
word = '14:14hrs'
word = multireplace(word, date_replace)
word

'14:14hrs'

In [33]:
dateparser.parse('14:14 hours', settings={'TIMEZONE': 'GMT+8'})

datetime.datetime(2022, 12, 3, 1, 35, 37, 152859)

In [19]:
normalizer._tokenizer(t)

['14:14hrs',
 'Kerja',
 'penyelenggaraan',
 'KM',
 '83.5',
 '-',
 'KM',
 '84.5',
 'arah',
 'Utara',
 'dari',
 'Ayer',
 'Hitam',
 'ke',
 'Yong',
 'Peng',
 '.',
 'Lorong',
 'kiri',
 'ditutup',
 '.',
 'Trafi',
 'https://t.co/iP2jEdY4Ib']

In [21]:
t

'14:14hrs Kerja penyelenggaraan KM 83.5- KM 84.5 arah Utara dari Ayer Hitam ke Yong Peng. Lorong kiri ditutup. Trafi https://t.co/iP2jEdY4Ib'